# RNN to generate text using Shakespeare's Sonnets as input
---

In [9]:
import torch
import torch.nn as nn
import numpy as np

#### RNN class

In [10]:
# Define the RNN architecture

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

Load data

In [11]:
!wget 'https://www.gutenberg.org/cache/epub/1041/pg1041.txt'

--2023-04-26 12:19:22--  https://www.gutenberg.org/cache/epub/1041/pg1041.txt
Risoluzione di www.gutenberg.org (www.gutenberg.org)... 152.19.134.47
Connessione a www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 119693 (117K) [text/plain]
Salvataggio in: «pg1041.txt.1»

pg1041.txt.1        100%[===================>] 116,89K   298KB/s    in 0,4s    

2023-04-26 12:19:24 (298 KB/s) - «pg1041.txt.1» salvato [119693/119693]



In [12]:
# pg1041.txt: Shakespeare' sonnets
with open('pg1041.txt', 'r') as f:
    text = f.read()

In [13]:
# Create a dictionary mapping each character to a unique index
chars = list(set(text))
char_to_idx = {ch:i for i, ch in enumerate(chars)}

In [25]:
# Define some hyperparameters
input_size = len(chars)
hidden_size = 128
output_size = len(chars)
learning_rate = 0.001

In [26]:
# Instantiate the RNN model
rnn = RNN(input_size, hidden_size, output_size)

In [27]:
# Define the loss function and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [43]:
# Train the RNN model

def train(input_tensor, target_tensor):
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i], hidden)
        loss += criterion(output, target_tensor[i])

    loss.backward()

    optimizer.step()

    return output, loss.item() / input_tensor.size()[0]

def char_tensor(text):
    tensor = torch.zeros(len(text), dtype=torch.long)
    for i, char in enumerate(text):
        tensor[i] = char_to_idx[char]
    return tensor

In [44]:
# Define the number of epochs to train for and the sequence length
n_epochs = 5000
seq_length = 100

# Train the RNN model for the specified number of epochs
for epoch in range(1, n_epochs + 1):
    # Generate a random sequence of characters from the input text
    start_idx = np.random.randint(0, len(text) - seq_length)
    end_idx = start_idx + seq_length + 1
    input_seq = text[start_idx:end_idx]
    target_seq = text[start_idx+1:end_idx+1]

    input_tensor = char_tensor(input_seq)
    target_tensor = char_tensor(target_seq)

    output, loss = train(input_tensor, target_tensor)

    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, n_epochs, loss))

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
# Generate text using the trained RNN model


def generate_text(start_string, length):
    with torch.no_grad():
        input_tensor = char_tensor(start_string)
        hidden = rnn.initHidden()

        output_text = start_string

        for i in range(length):
            output, hidden = rnn(input_tensor[i], hidden)

            # Sample the next character using the output probabilities
            probs = torch.exp(output)
            probs /= torch.sum(probs)
            char_idx = torch.multinomial(probs, 1)[0]

            # Add the predicted character to the output text
            char = chars[char_idx]
            output_text += char
            input_tensor = char_tensor(char)

        return output_text

In [ ]:
# Generate some sample text using the trained RNN model
start_string = 'shall i compare thee to a summer\'s day?\n'
generated_text = generate_text(start_string, 1000)
print(generated_text)